In [1]:
import numpy as np
import pickle
import keras.backend as K

from keras.layers import  GlobalAveragePooling1D, Input, Activation, MaxPooling1D, BatchNormalization, Dense, Dropout, Conv1D,GlobalMaxPooling1D
from keras.layers import GRU,AveragePooling1D,CuDNNGRU
from keras.layers.merge import Concatenate
from keras.models import Model 
from keras.callbacks import EarlyStopping,ModelCheckpoint
from gensim.models import Word2Vec 

import keras.backend.tensorflow_backend as KTF
import tensorflow as tf
import os


os.environ["CUDA_VISIBLE_DEVICES"] = "1"

config = tf.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.Session(config=config)

KTF.set_session(sess)

Using TensorFlow backend.
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/xhh/anaconda3/envs/py36/lib/p

In [2]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack
seed = 777
np.random.seed(seed)
torch.manual_seed(seed)

In [3]:
def gen_embedding(words4sent, max_seq_len, feature_dim,   to_reverse=0):
    length = []
    output = []
    
    for words in words4sent:
        if to_reverse:
            words = np.flip(words, 0)
        length.append( words.shape[0])
        if  words.shape[0] < max_seq_len:
            wordList = np.concatenate([words,np.zeros([max_seq_len - words.shape[0],feature_dim])])
        output.append(wordList)
    return np.array(output),np.array(length) 

In [4]:
def mean_pool(x, lengths):
    out = torch.FloatTensor(x.size(1), x.size(2)).zero_() # BxF
    for i in range(x.size(1)):
        out[i] = torch.mean(x[:lengths[i],i,:], 0)
    return out


class RandLSTM(nn.Module):

    def __init__(self, input_dim, num_layers, output_dim,  bidirectional=False):
        super(RandLSTM, self).__init__()
        

        self.bidirectional = bidirectional
        self.max_seq_len = 128
        self.input_dim = input_dim
         

        self.e_hid_init = torch.zeros(1, 1, output_dim)
        self.e_cell_init = torch.zeros(1, 1, output_dim)

        self.output_dim = output_dim
        self.num_layers = num_layers
        self.lm = nn.LSTM(input_dim, output_dim, num_layers=num_layers,
                          bidirectional= self.bidirectional, batch_first=True)

        self.bidirectional += 1
        
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

   

    def lstm(self, inputs, lengths):
        bsz, max_len, _ = inputs.size()
        in_embs = inputs
        lens, indices = torch.sort(lengths, 0, True)

        e_hid_init = self.e_hid_init.expand(1*self.num_layers*self.bidirectional, bsz, self.output_dim).contiguous()
        e_cell_init = self.e_cell_init.expand(1*self.num_layers*self.bidirectional, bsz, self.output_dim).contiguous()
        all_hids, (enc_last_hid, _) = self.lm(pack(in_embs[indices],
                                                        lens.tolist(), batch_first=True), (e_hid_init, e_cell_init))
        _, _indices = torch.sort(indices, 0)
        all_hids = unpack(all_hids, batch_first=True)[0][_indices]

        return all_hids

    def forward(self, words4sent):
        
        out, lengths = gen_embedding(words4sent, self.max_seq_len, self.input_dim)
        out = torch.from_numpy(out).float()
        lengths = torch.from_numpy(np.array(lengths))
        out = self.lstm(out, lengths)
#         print("output size:",out.size())
        out = out.transpose(1,0)
        out = mean_pool(out, lengths)
        return out

    def encode(self, batch):
        return self.forward(batch).cpu().detach().numpy()

In [5]:

from six.moves import cPickle as pickle #for performance

 
def save_dict(di_, filename_):
    with open(filename_, 'wb') as f:
        pickle.dump(di_, f)

def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pickle.load(f)
    return ret_di

In [6]:
# extract w2v model
w2vmodel =  Word2Vec.load('/home/xhh/PMC_model/PMC_model.txt')
def vector_name(name): 
    s = name.split(' ')
    vectors = [] 
    for word in s: 
        if w2vmodel.wv.__contains__(word):   
            vectors.append(w2vmodel.wv[word]) 
    else: 
        clear_words = re.sub('[^A-Za-z0-9]+', ' ', word) 
        clear_words = clear_words.lstrip().rstrip().split(' ')
        for w in clear_words: 
            if w2vmodel.wv.__contains__(w): 
                vectors.append(w2vmodel.wv[w]) 
    return vectors

In [7]:
# read go.obo obtain ontology type
id_type_dicts = {}
obo_file = '../cross-species/go.obo'
fp=open(obo_file,'r')
obo_txt=fp.read()
fp.close()
obo_txt=obo_txt[obo_txt.find("[Term]")-1:]
obo_txt=obo_txt[:obo_txt.find("[Typedef]")]
# obo_dict=parse_obo_txt(obo_txt)
id_name_dicts = {}
for Term_txt in obo_txt.split("[Term]\n"):
    if not Term_txt.strip():
        continue
    name = ''
    ids = []
    for line in Term_txt.splitlines():
        if   line.startswith("id: "):
            ids.append(line[len("id: "):])     
        elif line.startswith("name: "):
             name=line[len("name: "):]
        elif line.startswith("alt_id: "):
            ids.append(line[len("alt_id: "):])
    
    for t_id in ids:
        id_name_dicts[t_id] = name

In [8]:
import re
protein2go =  load_dict('yeast_data/protein2go_dicts.pkl')
prot2emb_w2v = {}
project_dim = 1024
num_layers = 1
max_go_len = 1024
max_protlen = 0
biLSTM_Flag = False
w2vlstm = RandLSTM(200,num_layers,  project_dim, bidirectional = biLSTM_Flag)


for key, value in protein2go.items(): 
    allgos = value.split(';') 
    allgos = list(set(allgos))
    count = 0
    words4sent = []
    for  go in  allgos:
        if len(go) > 2:
            feature = np.array(vector_name(id_name_dicts[go]))
            if feature.shape[0] > 0:
                words4sent.append(feature)
            
        count += feature.shape[0]
    if len(words4sent) > 0:
        sent_embedding = w2vlstm.encode(words4sent)
    else:
        if biLSTM_Flag:
            sent_embedding = np.zeros((1, project_dim*2))
        else:
            sent_embedding = np.zeros((1, project_dim))

    if max_protlen < sent_embedding.shape[0]:
        max_protlen = sent_embedding.shape[0]
    prot2emb_w2v[key] = sent_embedding 

del w2vmodel

In [9]:
print(max_protlen)
# w2v_len = max_protlen
w2v_len = 256
# w2v_len = 211

211


In [10]:
prot2emb_bert = {}
max_protlen = 0
input_dim = 768
 
bertlstm = RandLSTM(input_dim,num_layers,  project_dim, bidirectional = biLSTM_Flag)
for key, value in protein2go.items():
     
    allgos = value.split(';') 
    allgos = list(set(allgos))
    count = 0
    words4sent = []
    for  go in  allgos:
        if len(go) > 2:
            feature = np.load('../ncbi_allfeatures4go/'+go+'_0.npy')[1:-1]
            words4sent.append(feature)
        count += feature.shape[0] 
    if len(words4sent) > 0:
        sent_embedding = bertlstm.encode(words4sent)
    else:
        if biLSTM_Flag:
            sent_embedding = np.zeros((1, project_dim*2))
        else:
            sent_embedding = np.zeros((1, project_dim))

    if max_protlen < sent_embedding.shape[0]:
        max_protlen = sent_embedding.shape[0]
    prot2emb_bert[key] = sent_embedding 

In [11]:
max_protlen

211

In [12]:
# bert_len = max_protlen
bert_len = 256

In [13]:
# save_dict(prot2emb_bert, 'yeast_data/bert_sent_2048.pkl')
# save_dict(prot2emb_w2v, 'yeast_data/w2v_sent_2048.pkl')

In [14]:
import keras
feature_len = 768

max_seq_len = 1000
# max_protlen = 32
 




class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self,  ppi_pair_file, batch_size=128):
        'Initialization' 
        self.batch_size = batch_size
        self.ppi_pair_file = ppi_pair_file
        input_dim = 768
        num_layers = 1
        projection_dim = 2048
        if biLSTM_Flag:
            self.projection_dim = project_dim*2
        else:
            self.projection_dim = project_dim
        self.bert_len = bert_len
        self.w2v_len = w2v_len

        self.max_seqlen = max_seq_len  
        self.protein2seq = load_dict('yeast_data/protein2seq_dicts.pkl')
        self.read_ppi()
         
        self.prot2emb_bert =  prot2emb_bert
        self.prot2emb_w2v = prot2emb_w2v
#         self.prot2embedding() 
         
        self.on_epoch_end()
    
    def read_ppi(self):
        with open(self.ppi_pair_file, 'r') as f:
            self.ppi_pairs  =  f.readlines()
    
    
   

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.ppi_pairs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.ppi_pairs))
         
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization

         
         
        y = np.empty((self.batch_size))
        X_go1 = np.empty((self.batch_size, self.bert_len + self.w2v_len,self.projection_dim))
        X_go2 = np.empty((self.batch_size, self.bert_len + self.w2v_len,self.projection_dim))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            ppi_pair = self.ppi_pairs[ID]
            p1, p2, label = ppi_pair.rstrip().split('\t')
            if label == '1':
                y[i] = 1
            else:
                y[i] = 0
             
            
            prot1emb_bert = self.prot2emb_bert[p1]
            X_go1[i,:prot1emb_bert.shape[0]] = prot1emb_bert
            
            prot2emb_bert = self.prot2emb_bert[p2]
            X_go2[i,:prot2emb_bert.shape[0]] = prot2emb_bert
            
            prot1emb_w2v = self.prot2emb_w2v[p1]
            X_go1[i,prot1emb_bert.shape[0]:prot1emb_w2v.shape[0] + prot1emb_bert.shape[0]] = prot1emb_w2v
            
            prot2emb_w2v = self.prot2emb_w2v[p2]
            X_go2[i,prot2emb_bert.shape[0]:prot2emb_bert.shape[0] + prot2emb_w2v.shape[0] ] = prot2emb_w2v
            
         
            
            
        return [X_go1, X_go2] ,  y



    def all_data(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization

         
         
        y = np.empty((len(list_IDs_temp)))
        X_go1 = np.empty((len(list_IDs_temp), self.bert_len + self.w2v_len,self.projection_dim))
        X_go2 = np.empty((len(list_IDs_temp), self.bert_len + self.w2v_len,self.projection_dim))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            ppi_pair = self.ppi_pairs[ID]
            p1, p2, label = ppi_pair.rstrip().split('\t')
            if label == '1':
                y[i] = 1
            else:
                y[i] = 0
             
            
            prot1emb_bert = self.prot2emb_bert[p1]
            X_go1[i,:prot1emb_bert.shape[0]] = prot1emb_bert
            
            prot2emb_bert = self.prot2emb_bert[p2]
            X_go2[i,:prot2emb_bert.shape[0]] = prot2emb_bert
            
            prot1emb_w2v = self.prot2emb_w2v[p1]
            X_go1[i,prot1emb_bert.shape[0]:prot1emb_w2v.shape[0] + prot1emb_bert.shape[0]] = prot1emb_w2v
            
            prot2emb_w2v = self.prot2emb_w2v[p2]
            X_go2[i,prot2emb_bert.shape[0]:prot2emb_bert.shape[0] + prot2emb_w2v.shape[0] ] = prot2emb_w2v
            
         
           
        return [X_go1, X_go2] ,  y

In [15]:
from keras import backend as K, initializers, regularizers, constraints
from keras.engine.topology import Layer


def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        # todo: check that this is correct
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)


class Attention(Layer):
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True,
                 return_attention=False,
                 **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Note: The layer has been tested with Keras 1.x
        Example:
            # 1
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
            # 2 - Get the attention scores
            hidden = LSTM(64, return_sequences=True)(words)
            sentence, word_scores = Attention(return_attention=True)(hidden)
        """
        self.supports_masking = True
        self.return_attention = return_attention
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        eij = dot_product(x, self.W)

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        weighted_input = x * K.expand_dims(a)

        result = K.sum(weighted_input, axis=1)

        if self.return_attention:
            return [result, a]
        return result

    def compute_output_shape(self, input_shape):
        if self.return_attention:
            return [(input_shape[0], input_shape[-1]),
                    (input_shape[0], input_shape[1])]
        else:
            return input_shape[0], input_shape[-1]

In [16]:
from keras.layers import   Embedding
from keras.layers import  GRU, Bidirectional, CuDNNGRU, Lambda, Flatten
from keras.utils import multi_gpu_model
from keras.layers.merge import concatenate
from keras_radam import RAdam
from keras_lookahead import Lookahead

def inception_block(input_tensor, output_size):
    """"""
    con1d_filters = int(output_size/4)
    y = Conv1D(con1d_filters, 3, activation="relu", padding='same')(input_tensor)
    x1 = Conv1D(con1d_filters, 5, activation="relu", padding='same')(y)

    y = Conv1D(con1d_filters, 1, activation="relu", padding='valid')(input_tensor)
    x2 = Conv1D(con1d_filters, 3, activation="relu", padding='same')(y)

    x3 = Conv1D(con1d_filters, 3, activation="relu", padding='same')(input_tensor)
    x4 = Conv1D(con1d_filters, 1, activation="relu", padding='same')(input_tensor)

    y = Concatenate()([x1, x2, x3, x4])
#     y = MaxPooling1D(4)(mix0)
    # y = AveragePooling1D()(mix0)
#     y = BatchNormalization()(y)

    return y

class ResidualConv1D:
    """
    ***ResidualConv1D for use with best performing classifier***
    """

    def __init__(self, filters, kernel_size, pool=False):
        self.pool = pool
        self.kernel_size = kernel_size
        self.params = {
            "padding": "same",
            "kernel_initializer": "he_uniform",
            "strides": 1,
            "filters": filters,
        }

    def build(self, x):

        res = x
        if self.pool:
            x = MaxPooling1D(1, padding="same")(x)
            res = Conv1D(kernel_size=1, **self.params)(res)

        out = Conv1D(kernel_size=1, **self.params)(x)

#         out = BatchNormalization(momentum=0.9)(out)
        out = Activation("relu")(out)
        out = Conv1D(kernel_size=self.kernel_size, **self.params)(out)

#         out = BatchNormalization(momentum=0.9)(out)
        out = Activation("relu")(out)
        out = Conv1D(kernel_size=self.kernel_size, **self.params)(out)

        out = keras.layers.add([res, out])

        return out

    def __call__(self, x):
        return self.build(x)
def build_residual_cnn(input_x):
    

    x = Conv1D(
        filters=32,
        kernel_size=3,
        padding="same",
        kernel_initializer="he_uniform",
    )(input_x)
#     x = BatchNormalization()(x)
    x = Activation("relu")(x)

    # residual net part
    x = ResidualConv1D(filters=32, kernel_size=3, pool=True)(x)
    x = ResidualConv1D(filters=32, kernel_size=3)(x)
    x = ResidualConv1D(filters=32, kernel_size=3)(x)
    
    
#     x = ResidualConv1D(filters=32, kernel_size=3, pool=True)(x)
#     x = ResidualConv1D(filters=32, kernel_size=3)(x)
#     x = ResidualConv1D(filters=32, kernel_size=3)(x)
#     x = Activation("relu")(x)

    
#     x = ResidualConv1D(filters=32, kernel_size=3, pool=True)(x)
#     x = ResidualConv1D(filters=32, kernel_size=3)(x)
#     x = ResidualConv1D(filters=32, kernel_size=3)(x)
    
    x = Flatten()(x)
#     x = BatchNormalization(momentum=0.9)(x)
#     x = Activation("relu")(x)
#     x = MaxPooling1D(1, padding="same")(x)
#     x  = GlobalAveragePooling1D()(x)
    return x
    

def build_fc_model(input_x):
     
    
    x_a = GlobalAveragePooling1D()(input_x)
    x_b = GlobalMaxPooling1D()(input_x)
#     x_c = Attention()(input_x)
    x = Concatenate()([x_a, x_b])
    x = Dense(1024)(x)
#     x = Dense(256)(x)
    return x 


if biLSTM_Flag:
    input_dim = project_dim * 2
else:
    input_dim = project_dim
    
def build_model():
    con_filters = 128
    
    left_input_go = Input(shape=(bert_len+w2v_len,input_dim))
    right_input_go = Input(shape=(bert_len+w2v_len,input_dim))
    
    
    
    NUM_FILTERS = 32
    FILTER_LENGTH1 = 5
    FILTER_LENGTH2 = 5
    
    
    left_x_go= Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_LENGTH1,  activation='relu', padding='valid',  strides=1)(left_input_go)
    left_x_go = Conv1D(filters=NUM_FILTERS*2, kernel_size=FILTER_LENGTH1,  activation='relu', padding='valid',  strides=1)(left_x_go)
    left_x_go = Conv1D(filters=NUM_FILTERS*3, kernel_size=FILTER_LENGTH1,  activation='relu', padding='valid',  strides=1)(left_x_go)
    left_x_go_max = GlobalMaxPooling1D()(left_x_go) #pool_size=pool_length[i]
    left_x_go_avg = GlobalAveragePooling1D()(left_x_go) #pool_size=pool_length[i]


    right_x_go = Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_LENGTH2,  activation='relu', padding='valid',  strides=1)(right_input_go)
    right_x_go = Conv1D(filters=NUM_FILTERS*2, kernel_size=FILTER_LENGTH2,  activation='relu', padding='valid',  strides=1)(right_x_go)
    right_x_go = Conv1D(filters=NUM_FILTERS*3, kernel_size=FILTER_LENGTH2,  activation='relu', padding='valid',  strides=1)(right_x_go)
    right_x_go_max = GlobalMaxPooling1D()(right_x_go)
    right_x_go_avg = GlobalAveragePooling1D()(right_x_go)

    
    
#     left_x_go = build_residual_cnn(left_input_go)
#     right_x_go = build_residual_cnn(right_input_go)
     
     
    x =   Concatenate()([left_x_go_avg, left_x_go_max  , right_x_go_avg, right_x_go_max])
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(512, activation='relu')(x)
  
     
    x = Dense(1)(x)
    output = Activation('sigmoid')(x)
    # model = Model([left_input_go, right_input_go], output)
    optimizer = Lookahead(RAdam())
  
    model = Model([left_input_go, right_input_go], output)
#     model = multi_gpu_model(model, gpus=2)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

model = build_model()
print(model.summary())





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 1024)    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 512, 1024)    0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 508, 32)      163872      input_1[0][0]                    
___________________________________________________________________________

In [17]:
from sklearn.model_selection import StratifiedKFold
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from keras.utils import multi_gpu_model

dataset_name = 'yeast'
for rep in range(1):
    n_splits = 5
    TPRs =  np.zeros(n_splits)
    FPRs = np.zeros(n_splits)
    Precs = np.zeros(n_splits)
    ACCs = np.zeros(n_splits)
    F1s = np.zeros(n_splits)
    MCCs = np.zeros(n_splits)
    AUCs = np.zeros(n_splits)
     
    count = 0
    for split in range(n_splits):
        train_pairs_file = 'yeast_data/new_train_valid-'+str(split)
        test_pairs_file = 'yeast_data/new_test-'+str(split)
#         valid_pairs_file = 'yeast_data/new_valid'+str(split)

        batch_size = 128
        train_generator = DataGenerator(   train_pairs_file,batch_size = batch_size )
        test_generator = DataGenerator(   test_pairs_file,batch_size = batch_size)
#         valid_generator = DataGenerator(   valid_pairs_file,batch_size = batch_size)
         
        # model = build_model_without_att()
        model = build_model()
        save_model_name = 'yeast_data/fusion_sent_GoplusSeq'+str(rep)+'-'+str(split) + '.hdf5'
        
        earlyStopping = EarlyStopping(monitor='val_acc', patience=20, verbose=0, mode='max')
        save_checkpoint = ModelCheckpoint(save_model_name, save_best_only=True, monitor='val_acc', mode='max', save_weights_only=True)

         
        # validation_data = (valid_X, valid_Y),  verbose=1,callbacks=[earlyStopping, save_checkpoint]
        #  max_queue_size=16, workers=8, use_multiprocessing=True,
        # validation_data=valid_generator,callbacks=[earlyStopping, save_checkpoint] 
        hist = model.fit_generator(generator=train_generator,
                    epochs = 60,verbose=1)
         
        
        # model = load_model(save_model_name)
#         model.load_weights(save_model_name)
        with open(test_pairs_file, 'r') as f:
            test_ppi_pairs  =  f.readlines()

        test_len = len(test_ppi_pairs) 
        list_IDs_temp = np.arange(test_len)

        test_x, y_test = test_generator.all_data(list_IDs_temp)

        y_pred_prob = model.predict(test_x)

       
        y_pred = (y_pred_prob > 0.5)
        auc = metrics.roc_auc_score(y_test, y_pred_prob) 
        f1 = f1_score(y_test, y_pred)
        pre = precision_score(y_test, y_pred)
        acc = accuracy_score(y_test, y_pred)

        precision, recall, _thresholds = metrics.precision_recall_curve(y_test, y_pred_prob)
        pr_auc = metrics.auc(recall, precision)
        mcc = matthews_corrcoef(y_test, y_pred)

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        total=tn+fp+fn+tp
        sen = float(tp)/float(tp+fn)
        sps = float(tn)/float((tn+fp))

        tpr = float(tp)/float(tp+fn)
        fpr = float(fp)/float((tn+fp))
        print('--------------------------\n')
        print ('AUC: %f' % auc)
        print ('ACC: %f' % acc) 
        # print("PRAUC: %f" % pr_auc)
        print ('MCC : %f' % mcc)
        # print ('SEN: %f' % sen)
        # print ('SEP: %f' % sps)
        print('TPR:%f'%tpr)
        print('FPR:%f'%fpr)
        print('Pre:%f'%pre)
        print('F1:%f'%f1)
        print('--------------------------\n')
        TPRs[count] = tpr
        FPRs[count] = fpr
        Precs[count] =pre
        ACCs[count] =acc
        F1s[count] =f1
        MCCs[count] =mcc
        AUCs[count] =auc
        count += 1
        del test_x
        del y_test
    print ('mean AUC: %f' % np.mean(AUCs))
    print ('mean ACC: %f' % np.mean(ACCs)) 
    print ('mean MCC : %f' % np.mean(MCCs))
    print('mean TPR:%f'% np.mean(TPRs))
    print('mean FPR:%f'% np.mean(FPRs))
    print('mean Pre:%f'% np.mean(Precs))
    print('mean F1:%f'% np.mean(F1s))
    np.savez('fusion_sent_seq_and_go__incep_'+str(rep), AUCs=AUCs, ACCs=ACCs, MCCs=MCCs, TPRs = TPRs, FPRs=FPRs, Precs=Precs, F1s=F1s)
    
    
    
    
    

Epoch 1/60
69/69 [==============================] - 58s 835ms/step - loss: 0.6652 - acc: 0.6155
Epoch 2/60
69/69 [==============================] - 53s 767ms/step - loss: 0.6436 - acc: 0.6259
Epoch 3/60
69/69 [==============================] - 53s 765ms/step - loss: 0.5931 - acc: 0.6651
Epoch 4/60
69/69 [==============================] - 53s 766ms/step - loss: 0.5264 - acc: 0.7313
Epoch 5/60
69/69 [==============================] - 53s 766ms/step - loss: 0.4671 - acc: 0.7742
Epoch 6/60
69/69 [==============================] - 53s 763ms/step - loss: 0.4054 - acc: 0.8106
Epoch 7/60
69/69 [==============================] - 53s 764ms/step - loss: 0.3327 - acc: 0.8548
Epoch 8/60
69/69 [==============================] - 52s 756ms/step - loss: 0.3144 - acc: 0.8585
Epoch 9/60
69/69 [==============================] - 51s 743ms/step - loss: 0.2468 - acc: 0.8940
Epoch 10/60
69/69 [==============================] - 51s 739ms/step - loss: 0.2235 - acc: 0.9002
Epoch 11/60
69/69 [====================

69/69 [==============================] - 55s 794ms/step - loss: 0.1046 - acc: 0.9578
Epoch 25/60
69/69 [==============================] - 56s 805ms/step - loss: 0.1145 - acc: 0.9520
Epoch 26/60
69/69 [==============================] - 55s 804ms/step - loss: 0.0878 - acc: 0.9651
Epoch 27/60
69/69 [==============================] - 56s 805ms/step - loss: 0.0748 - acc: 0.9703
Epoch 28/60
69/69 [==============================] - 55s 797ms/step - loss: 0.0711 - acc: 0.9725
Epoch 29/60
69/69 [==============================] - 55s 795ms/step - loss: 0.0808 - acc: 0.9695
Epoch 30/60
69/69 [==============================] - 55s 798ms/step - loss: 0.0632 - acc: 0.9778
Epoch 31/60
69/69 [==============================] - 56s 806ms/step - loss: 0.0698 - acc: 0.9729
Epoch 32/60
69/69 [==============================] - 55s 804ms/step - loss: 0.0620 - acc: 0.9771
Epoch 33/60
69/69 [==============================] - 56s 806ms/step - loss: 0.0554 - acc: 0.9791
Epoch 34/60
69/69 [=======================

69/69 [==============================] - 52s 757ms/step - loss: 0.6495 - acc: 0.6756
Epoch 48/60
69/69 [==============================] - 57s 823ms/step - loss: 0.3841 - acc: 0.8222
Epoch 49/60
69/69 [==============================] - 57s 825ms/step - loss: 0.1466 - acc: 0.9421
Epoch 50/60
69/69 [==============================] - 57s 824ms/step - loss: 0.0671 - acc: 0.9750
Epoch 51/60
69/69 [==============================] - 57s 826ms/step - loss: 0.0652 - acc: 0.9771
Epoch 52/60
69/69 [==============================] - 57s 822ms/step - loss: 0.0697 - acc: 0.9754
Epoch 53/60
69/69 [==============================] - 53s 769ms/step - loss: 0.0554 - acc: 0.9795
Epoch 54/60
69/69 [==============================] - 53s 775ms/step - loss: 0.0428 - acc: 0.9852
Epoch 55/60
69/69 [==============================] - 56s 809ms/step - loss: 0.1467 - acc: 0.9470
Epoch 56/60
69/69 [==============================] - 56s 813ms/step - loss: 0.0531 - acc: 0.9811
Epoch 57/60
69/69 [=======================

69/69 [==============================] - 57s 821ms/step - loss: 0.3024 - acc: 0.8644
Epoch 9/60
69/69 [==============================] - 56s 817ms/step - loss: 0.2534 - acc: 0.8864
Epoch 10/60
69/69 [==============================] - 57s 827ms/step - loss: 0.2344 - acc: 0.8950
Epoch 11/60
69/69 [==============================] - 56s 813ms/step - loss: 0.2169 - acc: 0.9021
Epoch 12/60
69/69 [==============================] - 56s 813ms/step - loss: 0.3125 - acc: 0.8468
Epoch 13/60
69/69 [==============================] - 56s 814ms/step - loss: 0.2119 - acc: 0.9072
Epoch 14/60
69/69 [==============================] - 56s 818ms/step - loss: 0.1878 - acc: 0.9173
Epoch 15/60
69/69 [==============================] - 52s 759ms/step - loss: 0.1591 - acc: 0.9289
Epoch 16/60
69/69 [==============================] - 52s 757ms/step - loss: 0.1529 - acc: 0.9312
Epoch 17/60
69/69 [==============================] - 55s 795ms/step - loss: 0.1394 - acc: 0.9369
Epoch 18/60
69/69 [========================

In [18]:
# del test_x
# del y_test

In [19]:
model.load_weights(save_model_name)
with open(test_pairs_file, 'r') as f:
    test_ppi_pairs  =  f.readlines()

test_len = len(test_ppi_pairs) 
list_IDs_temp = np.arange(test_len)

test_x, y_test = test_generator.all_data(list_IDs_temp)

y_pred_prob = model.predict(test_x)


y_pred = (y_pred_prob > 0.5)
auc = metrics.roc_auc_score(y_test, y_pred_prob) 
f1 = f1_score(y_test, y_pred)
pre = precision_score(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)

precision, recall, _thresholds = metrics.precision_recall_curve(y_test, y_pred_prob)
pr_auc = metrics.auc(recall, precision)
mcc = matthews_corrcoef(y_test, y_pred)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
total=tn+fp+fn+tp
sen = float(tp)/float(tp+fn)
sps = float(tn)/float((tn+fp))

tpr = float(tp)/float(tp+fn)
fpr = float(fp)/float((tn+fp))
print('--------------------------\n')
print ('AUC: %f' % auc)
print ('ACC: %f' % acc) 
# print("PRAUC: %f" % pr_auc)
print ('MCC : %f' % mcc)
# print ('SEN: %f' % sen)
# print ('SEP: %f' % sps)
print('TPR:%f'%tpr)
print('FPR:%f'%fpr)
print('Pre:%f'%pre)
print('F1:%f'%f1)

ValueError: You are trying to load a weight file containing 74 layers into a model with 10 layers.